In [4]:
from astropy.table import QTable, hstack
import numpy as np
import time
import datetime
import itertools
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# !ls

In [6]:
path = r'/Users/amielpatton-hall/Desktop/Research/Larson/CLEAN3021/PYTHON/SLOAN/'  #defining location of Phot and trilegal files.
cluster = 'turner11'

# Phot_unidexed = QTable.read(path + cluster +'_SDSSRelease7.txt', format='ascii')
# Trilegal_unidexed = QTable.read(path + cluster +'_TRILEGAL.dat', format='ascii')

In [7]:
Phot_unidexed = QTable.read(cluster +'_SDSSRelease8.txt', format='ascii')   #you will need to adjust Release #

In [8]:
!cat turner11_TRILEGAL.dat | wc

  855485 14543229 96669696


In [9]:
# Trilegal_unidexed = QTable.read(cluster +'_TRILEGAL.dat', format='ascii',skip_footer=1)
file_ = cluster +'_TRILEGAL.dat'
print file_
trilegal_glob = glob.glob(file_)
with open(trilegal_glob[0]) as f_in:
    trilegal_unindexed = np.genfromtxt(f_in,skip_header=1,skip_footer=1,
                          dtype=[('Gc', '<f8'), ('[M/H]', '<f8'),('m_ini', '<f8'),('logL', '<f8'),
                                 ('logTe', '<f8'), ('logg', '<f8'),('m-M0', '<f8'),('Av', '<f8'),
                                 ('m2/m1', '<f8'), ('mbol', '<f8'),('u', '<f8'),('g', '<f8'),
                                ('r', '<f8'), ('i', '<f8'),('z', '<f8'),('Mact', '<f8'),])
#Gc logAge [M/H] m_ini   logL   logTe logg  m-M0   Av    m2/m1 mbol   u      g      r      i      z        Mact

turner11_TRILEGAL.dat


In [10]:
ID = QTable()
ID['id'] = range(len(Phot_unidexed)+1)[1:]
Phot = hstack([ID,Phot_unidexed],join_type='outer')
del ID

print '# stars given by SDSS          ',len(Phot)
print '# stars with g magnitudes      ',len(Phot[Phot['gmag']>=Phot['gmag'].min()])
print '# stars with g,r magnitudes    ',len(Phot[(Phot['gmag']>=Phot['gmag'].min())&(Phot['rmag']>=Phot['rmag'].min())])
print '# stars with g,r,i magnitudes  ',len(Phot[(Phot['imag']>=Phot['imag'].min())&(Phot['gmag']>=Phot['gmag'].min())&(Phot['rmag']>=Phot['rmag'].min())])
print '# stars with g,r,i,u magnitudes',len(Phot[(Phot['imag']>=Phot['imag'].min())&(Phot['gmag']>=Phot['gmag'].min())&(Phot['rmag']>=Phot['rmag'].min())&(Phot['umag']>=Phot['umag'].min())])
print '# stars removed                ',len(Phot) - len(Phot[(Phot['imag']>=Phot['imag'].min())&(Phot['gmag']>=Phot['gmag'].min())&(Phot['rmag']>=Phot['rmag'].min())&(Phot['umag']>=Phot['umag'].min())])

#to remove stars with missing magnitudes and redefine Phot without missing mags
Phot = Phot[(Phot['imag']>=Phot['imag'].min())&(Phot['gmag']>=Phot['gmag'].min())&(Phot['rmag']>=Phot['rmag'].min())&(Phot['umag']>=Phot['umag'].min())]

Phot['g-r'] = Phot['gmag']-Phot['rmag']
Phot['e_g-r']=(Phot['e_gmag']**2 + Phot['e_rmag']**2)**.5

# stars given by SDSS           3858
# stars with g magnitudes       3858
# stars with g,r magnitudes     3858
# stars with g,r,i magnitudes   3858
# stars with g,r,i,u magnitudes 3858
# stars removed                 0


In [11]:
Phot[:3]

id,_RAJ2000,_DEJ2000,mode,q_mode,cl,SDSS8,Im,m_SDSS8,RAJ2000,DEJ2000,zsp,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Q,ObsDate,g-r,e_g-r
int64,float64,float64,int64,float64,int64,str19,str2,str1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
1,310.743662,35.553875,1,1.0,6,J204258.47+353313.9,Im,--,310.743662,35.553875,--,23.748,0.74,23.619,0.33,22.166,0.124,21.128,0.08,20.903,0.226,3,2003.7184,1.453,0.352528013071
2,310.743702,35.564323,1,--,6,J204258.48+353351.5,Im,--,310.743702,35.564323,--,24.834,0.953,20.893,0.035,19.22,0.013,18.457,0.011,18.018,0.022,3,2003.7184,1.673,0.0373363094052
3,310.744023,35.548545,1,--,6,J204258.56+353254.7,Im,--,310.744023,35.548545,--,24.529,1.045,24.761,0.707,22.383,0.165,21.706,0.145,21.587,0.43,3,2003.7184,2.378,0.725998622588


In [12]:
Trilegal = QTable()
Trilegal['id'] = range(len(trilegal_unindexed)+1)[1:]
Trilegal['g'] = trilegal_unindexed['g']
Trilegal['r'] = trilegal_unindexed['r']
Trilegal['g-r'] = Trilegal['g'] - Trilegal['r']

In [13]:
Trilegal[:3]

id,g,r,g-r
int64,float64,float64,float64
1,33.979,30.206,3.773
2,34.812,30.07,4.742
3,20.406,18.698,1.708


In [14]:
# To define all needed values as arrays and build upper and lower y and b-y limits for each star:
id_phot = Phot['id'] #following renaming convention used in Clean3201
gmag_phot = Phot['gmag']
colgr_phot = Phot['g-r']
sigg_phot = Phot['e_gmag']
siggr_phot = Phot['e_g-r']

gmag_tri = Trilegal['g']
colgr_tri =  Trilegal['g-r']

n_stars = len(Phot)
n_f_stars = len(Trilegal)

ratio = 1

# y - determining ry
edge_g_phot = ratio*2 * sigg_phot
for i in range(n_stars):
    if .2 > edge_g_phot[i]:
        edge_g_phot[i] = ratio * .2
        
# b-y - determining rby
edge_gr_phot = ratio*2 * siggr_phot
for i in range(n_stars):
    if .1 > edge_gr_phot[i]:
        edge_gr_phot[i] = ratio * .1

# Final upper and lower limits determined:
u_limit_g_phot = gmag_phot + edge_g_phot
l_limit_g_phot = gmag_phot - edge_g_phot
u_limit_gr_phot = colgr_phot + edge_gr_phot
l_limit_gr_phot = colgr_phot - edge_gr_phot

print '# Phot    ',n_stars
print '# Trilegal',n_f_stars
# print id_phot[:3]
# print gmag_phot[:3]
# print colgr_phot[:3]
# print sigg_phot[:3]
# print siggr_phot[:3]
# print gmag_tri[:3]
# print colgr_tri[:3]
# print 'edge_g_phot:',edge_g_phot[:3]
# print 'edge_gr_phot:',edge_gr_phot[:3]
# print 'u_limit_g_phot:',u_limit_g_phot[:3]
# print 'l_limit_g_phot:',l_limit_g_phot[:3]
# print 'u_limit_gr_phot:',u_limit_gr_phot[:3]
# print 'l_limit_gr_phot:',l_limit_gr_phot[:3]

# Phot     3858
# Trilegal 855483


In [15]:
#Finding # local PHOT stars around all PHOT Stars
start_time_phot = time.time()

n = 20          #first run for 10 stars to find full run time
# n = n_stars     #then run for all stars
n_near_g_gr = []
for star in range(n):
    a = 0
    l_limit_g_phot_star = l_limit_g_phot[star]
    u_limit_g_phot_star = u_limit_g_phot[star]
    l_limit_gr_phot_star = l_limit_gr_phot[star]
    u_limit_gr_phot_star = u_limit_gr_phot[star]
    for near in range(n_stars):
        mag = gmag_phot[near]
        range_1 = (mag > l_limit_g_phot_star)
        if range_1 == True:
            range_3 = (mag < u_limit_g_phot_star)
            if range_3 == True:
                col = colgr_phot[near]
                range_2 = (col > l_limit_gr_phot_star)
                if range_2 == True:
                    range_4 = (col < u_limit_gr_phot_star)
                    if range_4 == True:
                        a += 1.0
    n_near_g_gr.append(a)

end_time_phot = time.time()

In [16]:
n_near_g_gr = np.array(n_near_g_gr, dtype='f')  #turning array into dtype float
backup_phot = n_near_g_gr
# n_near_g_gr = backup_phot   # use this line to use backup n_near instead of new n_near, make sure to # previous 2 lines

d_t1 = end_time_phot - start_time_phot

print "PHOT"
print '#:', np.array(n_near_g_gr, dtype='i'), '    mean:',n_near_g_gr.mean()
print '%:',n_near_g_gr/n_stars, '    mean:',(n_near_g_gr/n_stars).mean()
print "Iterations:      %g" % len(n_near_g_gr), "/ %g" % n_stars
print "Elapsed time:    %g seconds" % d_t1
print "Rate:            %g sec/range" % (d_t1/n)
print "Est Total time:  %g minutes" % (n_stars/n * d_t1/60), " / %g seconds" % (n_stars / n * d_t1), " / %g hours" % (n_stars/n * d_t1/3660) 

PHOT
#: [417  34 743  26 623 665 173 216 643  37  94 625  57  49  24  91   1  98
  66  48]     mean: 236.5
%: [ 0.10808709  0.00881286  0.19258684  0.00673924  0.16148263  0.17236911
  0.04484189  0.05598756  0.16666667  0.00959046  0.02436496  0.16200104
  0.01477449  0.01270088  0.00622084  0.02358735  0.0002592   0.02540176
  0.01710731  0.01244168]     mean: 0.0613012
Iterations:      20 / 3858
Elapsed time:    14.6098 seconds
Rate:            0.730491 sec/range
Est Total time:  46.7514 minutes  / 2805.09 seconds  / 0.766417 hours


In [17]:
#Finding # local TRILEGAL stars around all PHOT Stars
start_time_tri = time.time()

m = 10            #first run for 10 stars to find full run time
# m = n_stars         #then run for all stars
n_near_g_gr_tri = []
for star in range(m):
    a = 0
    l_limit_g_phot_star = l_limit_g_phot[star]
    u_limit_g_phot_star = u_limit_g_phot[star]
    l_limit_gr_phot_star = l_limit_gr_phot[star]
    u_limit_gr_phot_star = u_limit_gr_phot[star]
    for near in range(n_f_stars):
        mag = gmag_tri[near]
        range_1 = (mag > l_limit_g_phot_star)
        if range_1 == True:
            range_3 = (mag < u_limit_g_phot_star)
            if range_3 == True:
                col = colgr_tri[near]
                range_2 = (col > l_limit_gr_phot_star)
                if range_2 == True:
                    range_4 = (col < u_limit_gr_phot_star)
                    if range_4 == True:
                        a += 1.0
    n_near_g_gr_tri.append(a)
    
end_time_tri = time.time()

In [18]:
n_near_g_gr_tri = np.array(n_near_g_gr_tri, dtype='f')  #turning array into dtype float
backup_tri = n_near_g_gr_tri
# n_near_g_gr_tri = backup_tri   # use this line to use backup n_near instead of new n_near, make sure to # previous 2 lines

d_t2 = end_time_tri - start_time_tri

print "TRILEGAL"
print '#:', np.array(n_near_g_gr_tri, dtype='i'), '    mean:',n_near_g_gr_tri.mean()
print '%:',n_near_g_gr_tri/n_f_stars, '    mean:',(n_near_g_gr_tri/n_stars).mean()
print "Iterations:      %g" % len(n_near_g_gr_tri), "/ %g" % n_stars
print "Elapsed time:    %g seconds" % d_t2
print "Rate:            %g sec/range" % (d_t2/m)
print "Est Total time:  %g minutes" %(n_stars/m * d_t2/60), " / %g seconds" % (n_stars / m * d_t2), " / %g hours" % (n_stars/m * d_t2/3660)

TRILEGAL
#: [21166   944 78417  3369 44701 66496 15672    19 67722   940]     mean: 29944.6
%: [  2.47415787e-02   1.10347020e-03   9.16640074e-02   3.93812618e-03
   5.22523533e-02   7.77291892e-02   1.83194757e-02   2.22096757e-05
   7.91622978e-02   1.09879448e-03]     mean: 7.76169
Iterations:      10 / 3858
Elapsed time:    55.5837 seconds
Rate:            5.55837 sec/range
Est Total time:  356.662 minutes  / 21399.7 seconds  / 5.84691 hours


In [ ]:
#ONLY Refresh if completing different cycle and want new files

# t_backup = t                  #to use i_backup time replace i with i_backup in savetxt commands.
t = datetime.datetime.now()   #DATE + TIME specified in file name.

# print 'backup old time:',t_backup.isoformat()
print 'new time:       ',t.date()

In [ ]:
Limits = QTable()
Limits['id'] = id_phot
Limits['gmag'] = gmag_phot
Limits['colgr'] = colgr_phot
Limits['sigg'] = sigg_phot
Limits['siggr'] = colgr_phot
Limits['edge_g'] = edge_g_phot
Limits['edge_gr'] = edge_gr_phot
Limits['u_limit_g'] = u_limit_g_phot
Limits['l_limit_g'] = l_limit_g_phot
Limits['u_limit_gr'] = u_limit_gr_phot
Limits['l_limit_gr'] = l_limit_gr_phot
Limits['n_near_phot'] = n_near_g_gr            # variable NGC - phot
Limits['n_near_trilegal'] = n_near_g_gr_tri    # variable NOFF - trilegal
np.savetxt(cluster +'_'+ str(t.date())+'_backup_limits'+'.txt', Limits, delimiter=',', fmt='%s', header='id gmag colgr sigg siggr edge_g edge_gr u_limit_g l_limit_g u_limit_gr l_limit_gr n_near_phot n_near_trilegal')
print Limits[:3]

In [ ]:
alpha=0.38

ngc = np.array(n_near_g_gr, dtype='f')     # NGC   Phot
NOFF = np.array(n_near_g_gr_tri, dtype='f') # NOFF  trilegal

glob95 = ngc*((1-(1/(9*ngc))-(1.645/(3*np.sqrt((ngc))))+(0.031*(ngc**(-2.5))))**3)
field84 = (NOFF+1)*((1.0-1.0/(9.*(NOFF+1)))+( 1./(3.*np.sqrt(NOFF+1))))**3   

val = (alpha*field84)/glob95
for i in range(len(val)):
    if val[i] > 1.0:
          val[i]=1.0

p = 1 - val

In [ ]:
def random(ix,iy,iz):
    random = []
    for i in range(len(ix)):
        x = ix[i]
        y = iy[i]
        z = iz[i]
        x = 171 * (x%177) -  2 * (x/177)
        y = 171 * (y%176) - 35 * (y/176)
        z = 170 * (z%178) - 63 * (z/178)

        if x < 0:
            x += 30269
        if y < 0:
            y += 30307
        if z < 0:
            z += 30323
        random.append((x/30269. + y/30307. + z/30323.) % 1.0)
    return random

In [ ]:
ix=abs(100*field84)
iy=abs(200*glob95)
iz=abs(50*val)

prand=random(ix,iy,iz)

k = 0
for i in range(n_stars):
    if NOFF[i] == 0:
        p[i] = 1.000
        prand[i] = 0.0
        k+=1
        
print "number of stars corrected", k

member = np.zeros(n_stars)
k = 0
for i in range(n):
    if prand[i] < p[i]:
        member[i] = 1
        k+=1
    else:
        member[i] = 0
        
print '# members', k
print '# non-members', n_stars-k
print '# total stars', n_stars

In [ ]:
Probability = QTable()
Probability['id'] = id_phot
Probability['NGC'] = n_near_g_gr     # NGC   Phot
Probability['NOFF'] = n_near_g_gr_tri # NOFF  trilegal
Probability['glob95'] = glob95
Probability['field84'] = field84
Probability['val'] = val
Probability['p'] = 1 - val
Probability['member'] = member
np.savetxt(cluster + '_' + str(t.date()) +'_backup_probability'+'.txt', Probability, delimiter=',', fmt='%s', header='id NGC NOFF glob95 field84 val p member')
print Probability[:3]

In [ ]:
Phot['p'] = p
Phot['member'] = member

fieldstars = Phot[np.where(member == 0)[0]]
outcluster = Phot[np.where(member == 1)[0]]
out3sig = Phot[np.where((member == 1) & (p >.95))[0]]

fieldstars.write(cluster +'_'+ str(t.date()) + '_fieldstars.txt', format='ascii')
outcluster.write(cluster +'_'+ str(t.date()) + '_outcluster.txt', format='ascii')
out3sig.write(cluster +'_'+ str(t.date()) + '_out3sig.txt', format='ascii')

print cluster + ' outcluster   '
print '  n stars  ', len(outcluster), '/', n_stars
print '  avg p    ', outcluster['p'].mean()
print cluster + ' out3sig      '
print '  n stars  ', len(out3sig), '/', n_stars
print '  avg p    ', out3sig['p'].mean()
print cluster + ' fieldstars   '
print '  n stars: ', len(fieldstars), '/', n_stars
print '  avg p:   ', fieldstars['p'].mean()